In [1]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": 51.5085,
    "longitude": -0.1257,
    "hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "wind_speed_10m", "wind_direction_10m", "wind_gusts_10m"],
    "temperature_unit": "fahrenheit",
    "wind_speed_unit": "ms",
    "timezone": "auto",
    "forecast_days": 7
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(3).ValuesAsNumpy()
hourly_wind_direction_10m = hourly.Variables(4).ValuesAsNumpy()
hourly_wind_gusts_10m = hourly.Variables(5).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
    start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
    end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
    freq = pd.Timedelta(seconds = hourly.Interval()),
    inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["dew_point_2m"] = hourly_dew_point_2m
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m

hourly_dataframe = pd.DataFrame(data = hourly_data)

# Save to CSV
csv_filename = "london_hourly_weather.csv"
hourly_dataframe.to_csv(csv_filename, index=False)
print(f"CSV saved: {csv_filename}")

print(hourly_dataframe)

Coordinates 51.5°N -0.12000012397766113°E
Elevation 23.0 m asl
Timezone b'Europe/London' None
Timezone difference to GMT+0 0 s
CSV saved: london_hourly_weather.csv
                         date  temperature_2m  relative_humidity_2m  \
0   2025-02-19 00:00:00+00:00       37.103001                  70.0   
1   2025-02-19 01:00:00+00:00       37.013000                  68.0   
2   2025-02-19 02:00:00+00:00       36.923000                  67.0   
3   2025-02-19 03:00:00+00:00       36.743000                  69.0   
4   2025-02-19 04:00:00+00:00       36.563000                  71.0   
..                        ...             ...                   ...   
163 2025-02-25 19:00:00+00:00       44.255299                  67.0   
164 2025-02-25 20:00:00+00:00       42.545300                  71.0   
165 2025-02-25 21:00:00+00:00       41.015297                  75.0   
166 2025-02-25 22:00:00+00:00       39.665298                  80.0   
167 2025-02-25 23:00:00+00:00       38.585300          

In [2]:
import pandas as pd
fut_df = pd.read_csv(csv_filename)

In [3]:
fut_df.head()

,date,temperature_2m,relative_humidity_2m,dew_point_2m,wind_speed_10m,wind_direction_10m,wind_gusts_10m
0,2025-02-19 00:00:00+00:00,37.103,70.0,28.223970,2.801785,87.954640,6.0
1,2025-02-19 01:00:00+00:00,37.013,68.0,27.431843,2.601922,87.797450,6.0
2,2025-02-19 02:00:00+00:00,36.923,67.0,26.986053,2.901724,88.025116,6.0
3,2025-02-19 03:00:00+00:00,36.743,69.0,27.528103,2.700000,90.000000,6.1
4,2025-02-19 04:00:00+00:00,36.563,71.0,28.050882,2.800000,90.000000,6.0


In [4]:
fut_df.rename(columns={'date': 'Time'}, inplace=True)
fut_df['Time'] = pd.to_datetime(fut_df['Time'])
fut_df['Year']=fut_df['Time'].dt.year
fut_df['Month']=fut_df['Time'].dt.month

In [5]:
fut_df['Hour'] = fut_df['Time'].dt.hour

In [6]:
fut_df

,Time,temperature_2m,relative_humidity_2m,dew_point_2m,wind_speed_10m,wind_direction_10m,wind_gusts_10m,Year,Month,Hour
0,2025-02-19 00:00:00+00:00,37.103000,70.0,28.223970,2.801785,87.954640,6.0,2025,2,0
1,2025-02-19 01:00:00+00:00,37.013000,68.0,27.431843,2.601922,87.797450,6.0,2025,2,1
2,2025-02-19 02:00:00+00:00,36.923000,67.0,26.986053,2.901724,88.025116,6.0,2025,2,2
3,2025-02-19 03:00:00+00:00,36.743000,69.0,27.528103,2.700000,90.000000,6.1,2025,2,3
4,2025-02-19 04:00:00+00:00,36.563000,71.0,28.050882,2.800000,90.000000,6.0,2025,2,4
...,...,...,...,...,...,...,...,...,...,...
163,2025-02-25 19:00:00+00:00,44.255300,67.0,33.990050,1.392839,248.962420,4.5,2025,2,19
164,2025-02-25 20:00:00+00:00,42.545300,71.0,33.803505,1.664332,237.264770,4.0,2025,2,20
165,2025-02-25 21:00:00+00:00,41.015297,75.0,33.699253,1.780449,231.842730,3.6,2025,2,21
166,2025-02-25 22:00:00+00:00,39.665300,80.0,34.003770,1.838477,224.999900,3.6,2025,2,22


In [7]:
import numpy as np
min_year = fut_df['Year'].min()
max_year = fut_df['Year'].max()
fut_df['Month_sin'] = np.sin(2 * np.pi * fut_df['Month'] / 12)
fut_df['Month_cos'] = np.cos(2 * np.pi * fut_df['Month'] / 12)

In [8]:
fut_df['Day'] = fut_df['Time'].dt.day
fut_df['Day_sin'] = np.sin(2 * np.pi * fut_df['Day'] / 31) 
fut_df['Day_cos'] = np.cos(2 * np.pi * fut_df['Day'] / 31)
fut_df['Hour_sin'] = np.sin(2 * np.pi * fut_df['Hour'] / 24) 
fut_df['Hour_cos'] = np.cos(2 * np.pi * fut_df['Hour'] / 24)

In [9]:
fut_df.head()

,Time,temperature_2m,relative_humidity_2m,dew_point_2m,wind_speed_10m,wind_direction_10m,wind_gusts_10m,Year,Month,Hour,Month_sin,Month_cos,Day,Day_sin,Day_cos,Hour_sin,Hour_cos
0,2025-02-19 00:00:00+00:00,37.103,70.0,28.223970,2.801785,87.954640,6.0,2025,2,0,0.866025,0.5,19,-0.651372,-0.758758,0.000000,1.000000
1,2025-02-19 01:00:00+00:00,37.013,68.0,27.431843,2.601922,87.797450,6.0,2025,2,1,0.866025,0.5,19,-0.651372,-0.758758,0.258819,0.965926
2,2025-02-19 02:00:00+00:00,36.923,67.0,26.986053,2.901724,88.025116,6.0,2025,2,2,0.866025,0.5,19,-0.651372,-0.758758,0.500000,0.866025
3,2025-02-19 03:00:00+00:00,36.743,69.0,27.528103,2.700000,90.000000,6.1,2025,2,3,0.866025,0.5,19,-0.651372,-0.758758,0.707107,0.707107
4,2025-02-19 04:00:00+00:00,36.563,71.0,28.050882,2.800000,90.000000,6.0,2025,2,4,0.866025,0.5,19,-0.651372,-0.758758,0.866025,0.500000


In [10]:
""" import pickle
with open('random_forest_model.pkl', 'rb') as f:
    model = pickle.load(f)
 """
import pickle
with open("random_forest_model.pkl", "rb") as f:
    model = pickle.load(f)

In [11]:
fut_df.rename(columns={"relative_humidity_2m":"relativehumidity_2m", "dew_point_2m":"dewpoint_2m", "wind_speed_10m":"windspeed_10m", "wind_direction_10m":"winddirection_10m", "wind_gusts_10m":"windgusts_10m"}, inplace=True)

In [12]:
pred_df = fut_df.drop(columns=["Time","Day","Year","Month","Hour"])
y_pred = model.predict(pred_df)

In [13]:
y_pred

array([0.26158433, 0.25113167, 0.292849  , 0.26399967, 0.28081733,
       0.24445667, 0.24091267, 0.179273  , 0.1573475 , 0.14867788,
       0.133021  , 0.14977833, 0.17191186, 0.14995133, 0.28177833,
       0.28652267, 0.29552067, 0.336115  , 0.28602733, 0.2957607 ,
       0.27102232, 0.30757052, 0.27134663, 0.38597644, 0.35745937,
       0.49893233, 0.68086633, 0.65744133, 0.752834  , 0.643236  ,
       0.63710933, 0.720949  , 0.70247433, 0.64927667, 0.80585167,
       0.74401433, 0.77047867, 0.84815856, 0.733154  , 0.81949067,
       0.85490367, 0.82544589, 0.80798819, 0.78746067, 0.77547133,
       0.777949  , 0.60987667, 0.579084  , 0.609259  , 0.44614967,
       0.373537  , 0.59087   , 0.621525  , 0.78709733, 0.72608987,
       0.73344556, 0.751067  , 0.80366233, 0.80002467, 0.743471  ,
       0.78247253, 0.80175367, 0.79503567, 0.8248992 , 0.83740167,
       0.79979473, 0.79217133, 0.79722333, 0.79397364, 0.75918   ,
       0.761341  , 0.67963067, 0.652587  , 0.58499467, 0.50577

'normalized_efficiency_plot.png'

In [14]:
import numpy as np
import matplotlib.pyplot as plt

efficiency_values = y_pred

daily_efficiency = efficiency_values.reshape(-1, 24).sum(axis=1)
normalized_efficiency = daily_efficiency / daily_efficiency.max()  # Normalize

dates = [f"Feb {i}" for i in range(8, 15)]

plt.figure(figsize=(10, 5))
plt.plot(dates, normalized_efficiency, marker='o', linestyle='-', color='b', label="Normalized Efficiency")

for i, txt in enumerate(normalized_efficiency):
    plt.text(dates[i], normalized_efficiency[i], f"{txt:.2f}", ha='right', va='bottom', fontsize=10)

plt.xlabel("Date")
plt.ylabel("Normalized Efficiency")
plt.title("Daily Normalized Efficiency from 8 Feb to 14 Feb")
plt.grid(True)
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()

# Save the figure
image_path = "normalized_efficiency_plot.png"
plt.savefig(image_path, dpi=300)
plt.show()

image_path


In [15]:
df_efficiency = pd.DataFrame({
    "Date": dates,
    "Normalized_Efficiency": normalized_efficiency
})

# Save to CSV
csv_path = "efficiency_data.csv"
df_efficiency.to_csv(csv_path, index=False)

# Display the table
print(df_efficiency)

csv_path

     Date  Normalized_Efficiency
0   Feb 8               0.305915
1   Feb 9               0.877073
2  Feb 10               0.896178
3  Feb 11               0.545756
4  Feb 12               1.000000
5  Feb 13               0.770903
6  Feb 14               0.238760


'efficiency_data.csv'

In [16]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": 51.5085,
    "longitude": -0.1257,
    "hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "wind_speed_10m", "wind_direction_10m", "wind_gusts_10m"],
    "temperature_unit": "fahrenheit",
    "wind_speed_unit": "ms",
    "timezone": "auto",
    "forecast_days": 7
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(3).ValuesAsNumpy()
hourly_wind_direction_10m = hourly.Variables(4).ValuesAsNumpy()
hourly_wind_gusts_10m = hourly.Variables(5).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
    start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
    end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
    freq = pd.Timedelta(seconds = hourly.Interval()),
    inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["dew_point_2m"] = hourly_dew_point_2m
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m

hourly_dataframe = pd.DataFrame(data = hourly_data)

# Save to CSV
csv_filename = "london_hourly_weather.csv"
hourly_dataframe.to_csv(csv_filename, index=False)
print(f"CSV saved: {csv_filename}")

print(hourly_dataframe)

Coordinates 51.5°N -0.12000012397766113°E
Elevation 23.0 m asl
Timezone b'Europe/London' None
Timezone difference to GMT+0 0 s
CSV saved: london_hourly_weather.csv
                         date  temperature_2m  relative_humidity_2m  \
0   2025-02-19 00:00:00+00:00       37.103001                  70.0   
1   2025-02-19 01:00:00+00:00       37.013000                  68.0   
2   2025-02-19 02:00:00+00:00       36.923000                  67.0   
3   2025-02-19 03:00:00+00:00       36.743000                  69.0   
4   2025-02-19 04:00:00+00:00       36.563000                  71.0   
..                        ...             ...                   ...   
163 2025-02-25 19:00:00+00:00       44.255299                  67.0   
164 2025-02-25 20:00:00+00:00       42.545300                  71.0   
165 2025-02-25 21:00:00+00:00       41.015297                  75.0   
166 2025-02-25 22:00:00+00:00       39.665298                  80.0   
167 2025-02-25 23:00:00+00:00       38.585300          